In [1]:
from decouple import config

In [3]:
from aimped.utils import download_model


In [21]:

# Örnek kullanım
result = download_model(
	bucket_name=config("BUCKET_NAME"),
	s3_folder=config("S3_FOLDER"),
	aws_access_key_id=config("AWS_ACCESS_KEY_ID"),
	aws_secret_access_key=config("AWS_SECRET_ACCESS_KEY"),
	local_dir="model_data",
	force_download=True
)
print(result)

NameError: name 'download_model' is not defined

In [4]:
bucket_name = config("BUCKET_NAME")
s3_folder=config("S3_FOLDER")
s3_folder


'aimped/nlp-embeddings-jina-small-en'

In [5]:
import re
import os
import base64

def determine_data_source(data):
    # S3 URI kontrolü (s3:// ile başlar)
    if isinstance(data, str) and data.startswith("s3://"):
        return "s3 uri"
    
    # URL kontrolü (http:// veya https:// ile başlar)
    url_pattern = re.compile(r'^(http://|https://)')
    if isinstance(data, str) and url_pattern.match(data):
        return "url"
    
    # Local path kontrolü (dosya mevcutsa)
    if isinstance(data, str) and os.path.exists(data):
        return "local_path"
    
    # Base64 kontrolü (Base64 string olup olmadığını kontrol eder)
    try:
        if isinstance(data, str):
            # Base64 string boşluk ve newline karakterlerinden temizlenir
            base64_bytes = base64.b64decode(data, validate=True)
            if base64_bytes:
                return "base64"
    except Exception:
        pass
    
    return "plain_text"

# # Örnek kullanım:
# data_list = [
#     "s3://bucket-name/file.txt",
#     "https://example.com/image.png",
#     "/mnt/sdb1/tasks/aimped-rev/notebooks/test.ipynb",
#     "aGVsbG8gd29ybGQ=",
#     "invalid data"
# ]

# results = [determine_data_source(data) for data in data_list]
# print(results)


['s3 uri', 'url', 'local_path', 'base64', 'plain_text']


In [20]:
import os
import base64
import boto3
import requests
import uuid
from aimped.s3_file_manager import S3FileManager
file_manager = S3FileManager( aws_access_key_id=config("AWS_ACCESS_KEY_ID"),
	                aws_secret_access_key=config("AWS_SECRET_ACCESS_KEY"),
                    region_name="us-east-1")

def file_type(file_path):
    return os.path.splitext(file_path)[1]

def process_payload(input_data:str, data_type=None):
    

    if not os.path.exists("input_data_folder"):
        os.makedirs("input_data_folder")
    
    
    uniq_id = str(uuid.uuid4())
    
    file_extension = file_type(input_data)
    
    valid_data_types = ["data_char", "data_pdf", "data_txt", "data_file", "data_json",
                        "data_image", "data_audio", "data_dicom", "data_svg"]
    if not data_type:
        raise ValueError("data_type is required")
    elif data_type not in valid_data_types:
        raise ValueError(f"Invalid data_type. Must be one of {valid_data_types}")   
    
    if not isinstance(input_data, str):
        raise ValueError("input_data must be a string")
    
    if data_type == "data_json":
        data_source = determine_data_source(input_data)
        if data_source == "plain_text":
            # control plain_text whether it is a local path or not
            if os.path.exists(input_data):
                raise ValueError("data_json cannot be a file path. Only plain text is allowed.")
            else:
                return input_data
        elif data_source == "s3 uri":
            raise ValueError("data_json cannot be a S3 URI. Only plain text is allowed.")
        elif data_source == "url":
            raise ValueError("data_json cannot be a URL. Only plain text is allowed.")
        elif data_source == "local_path":
            raise ValueError("data_json cannot be a local path. Only plain text is allowed.")
        elif data_source == "base64":
            raise ValueError("data_json cannot be a base64 string. Only plain text is allowed.")
        
    elif data_type == "data_char":
        pass # Todo: data_char için işlem yapılacak
    
    elif data_type == "data_pdf":
        
        data_source = determine_data_source(input_data)
        
        if file_extension != ".pdf":
            raise ValueError("data_pdf must be a PDF file")
        
        if data_source == "plain_text": # 1
            # control plain_text whether it is a local path or not
            if os.path.exists(input_data):
                return input_data
            else:
                raise ValueError("data_pdf must be a file path or S3 URI or URL or base64 string.")
        elif data_source == "s3 uri": # 2
            return file_manager.download_file_from_s3(config("BUCKET_NAME"), input_data,f"input_data_folder/{uniq_id}{file_extension}")
        elif data_source == "url": # 3
            response = requests.get(input_data)
            with open(f"input_data_folder/{uniq_id}{file_extension}", "wb") as f:
                f.write(response.content)
            return f"input_data_folder/{uniq_id}{file_extension}"
        elif data_source == "local_path": # 4
            return input_data
        elif data_source == "base64": # 5
            base64_bytes = base64.b64decode(input_data, validate=True)
            with open(f"input_data_folder/{uniq_id}{file_extension}", "wb") as f:
                f.write(base64_bytes)
            return f"input_data_folder/{uniq_id}{file_extension}"
        else:
            raise ValueError("data_pdf must be a file path or S3 URI or URL or base64 string")
    
    elif data_type == "data_txt":
        
        data_source = determine_data_source(input_data)
        
        if file_extension != ".txt":
            raise ValueError("data_txt must be a TXT file")
        
        if data_source == "plain_text":
            # control plain_text whether it is a local path or not
            if os.path.exists(input_data):
                return input_data
            else:
                raise ValueError("data_txt must be a file path or S3 URI or URL")
        elif data_source == "s3 uri":
            return file_manager.download_file_from_s3(config("BUCKET_NAME"), input_data,f"input_data_folder/{uniq_id}{file_extension}")
        elif data_source == "url":
            response = requests.get(input_data)
            with open(f"input_data_folder/{uniq_id}{file_extension}", "wb") as f:
                f.write(response.content)
            return f"input_data_folder/{uniq_id}{file_extension}"
        elif data_source == "local_path":
            return input_data
        else:
            raise ValueError("data_txt must be a file path or S3 URI or URL")
    
    
    elif data_type == "data_file":
        pass # Todo: data_file için işlem yapılacak
    
    elif data_type == "data_image":
        valid_img_extensions = [".jpg", ".jpeg", ".png"]
        data_source = determine_data_source(input_data)
        
        if file_extension.lower() not in valid_img_extensions:
            raise ValueError("data_image must be a JPG, JPEG or PNG file")
        
        if data_source == "plain_text":
            # control plain_text whether it is a local path or not
            if os.path.exists(input_data):
                return input_data
            else:
                raise ValueError("data_image must be a file path or S3 URI or URL or base64 string")
        elif data_source == "s3 uri":
            return file_manager.download_file_from_s3(config("BUCKET_NAME"), input_data,f"input_data_folder/{uniq_id}{file_extension}")
        elif data_source == "url":
            response = requests.get(input_data)
            with open(f"input_data_folder/{uniq_id}{file_extension}", "wb") as f:
                f.write(response.content)
            return f"input_data_folder/{uniq_id}{file_extension}"
        elif data_source == "local_path":   
            return input_data
        elif data_source == "base64":
            base64_bytes = base64.b64decode(input_data, validate=True)
            with open(f"input_data_folder/{uniq_id}{file_extension}", "wb") as f:
                f.write(base64_bytes)
            return f"input_data_folder/{uniq_id}{file_extension}"
        else:
            raise ValueError("data_image must be a file path or S3 URI or URL or base64 string")
    
    elif data_type == "data_audio":
        valid_audio_extensions = [".wav", ".mp3", ".mp4"]
        data_source = determine_data_source(input_data)
        
        if file_extension.lower() not in valid_audio_extensions:
            raise ValueError("data_audio must be a WAV, MP3 or MP4 file")
        
        if data_source == "plain_text":
            # control plain_text whether it is a local path or not
            if os.path.exists(input_data):
                return input_data
            else:
                raise ValueError("data_audio must be a file path or S3 URI or URL or base64 string")
        elif data_source == "s3 uri":
            return file_manager.download_file_from_s3(config("BUCKET_NAME"), input_data,f"input_data_folder/{uniq_id}{file_extension}")
        elif data_source == "url":
            response = requests.get(input_data)
            with open(f"input_data_folder/{uniq_id}{file_extension}", "wb") as f:
                f.write(response.content)
            return f"input_data_folder/{uniq_id}{file_extension}"
        elif data_source == "local_path":
            return input_data
        elif data_source == "base64":
            base64_bytes = base64.b64decode(input_data, validate=True)
            with open(f"input_data_folder/{uniq_id}{file_extension}", "wb") as f:
                f.write(base64_bytes)
            return f"input_data_folder/{uniq_id}{file_extension}"
        else:
            raise ValueError("data_audio must be a file path or S3 URI or URL or base64 string")
        
    elif data_type == "data_dicom":
        data_source = determine_data_source(input_data)
        if file_extension != ".dcm":
            raise ValueError("data_dicom must be a DCM file")
        
        if data_source == "plain_text":
            # control plain_text whether it is a local path or not
            if os.path.exists(input_data):
                return input_data
            else:
                raise ValueError("data_dicom must be a file path or S3 URI or URL or base64 string")
        elif data_source == "s3 uri":
            return file_manager.download_file_from_s3(config("BUCKET_NAME"), input_data,f"input_data_folder/{uniq_id}{file_extension}")
        elif data_source == "url":
            response = requests.get(input_data)
            with open(f"input_data_folder/{uniq_id}{file_extension}", "wb") as f:
                f.write(response.content)
            return f"input_data_folder/{uniq_id}{file_extension}"
        elif data_source == "local_path":
            return input_data
        elif data_source == "base64":
            base64_bytes = base64.b64decode(input_data, validate=True)
            with open(f"input_data_folder/{uniq_id}{file_extension}", "wb") as f:
                f.write(base64_bytes)
            return f"input_data_folder/{uniq_id}{file_extension}"
        else:
            raise ValueError("data_dicom must be a file path or S3 URI or URL or base64 string")
    
    elif data_type == "data_svg":
        pass # Todo: data_svg için işlem yapılacak
    
    else:
        raise ValueError("Invalid data_type")
        
            
            
            
            


In [22]:
process_payload("/mnt/sdb1/tasks/aimped-rev/notebooks/input_data_folder/1cef5d61-4a1a-4bcd-8a3a-ded961ac24eb.jpg", "data_json")

ValueError: data_json cannot be a local path. Only plain text is allowed.